
# Semantische Suche in arXiv-Artikeln mit LangChain

In dieser Übung verwenden wir arXiv, um echte wissenschaftliche Abstracts abzurufen, sie zu chunken und in einer Vektordatenbank zu speichern; und schließlich darauf semantische Suchen durchzuführen.

Dabei lernen Sie: 
- arXiv-Abstracts abzurufen und zu verarbeiten  
- Texte in Chunks aufzteilen  
- Embeddings zu erzeugen   
- eine Vektordatenbank aufbauen (Chroma)  
- semantische Ähnlichkeitssuche durchzuführen  


## arXiv
arXiv (ausgesprochen **archive**) ist eine Plattform für wissenschaftliche Preprints.

Forschende aus Bereichen wie Physik, Informatik, Mathematik, KI oder Statistik veröffentlichen dort ihre Arbeiten, bevor sie in Fachzeitschriften erscheinen.

- offen, kostenlos und ohne Login nutzbar.
- strukturierte Metadaten (Titel, Abstract, Autor\*innen, Veröffentlichungsdatum)
- API-Zugang, dadurch nutzbar für Text- und Datenanalysen


## LangChain
LangChain ist ein Python-Framework zur Entwicklung von Anwendungen, die mit großen Sprachmodellen (LLMs) oder semantischer Textverarbeitung arbeiten.
Es stellt Bausteine bereit für:

- Datenverarbeitung: Laden, Aufteilen und Strukturieren von Texten
- Embeddings: Umwandlung von Text in numerische Vektoren zur semantischen Analyse
- Vektordatenbanken: Speicherung und Wiederfinden semantisch ähnlicher Inhalte
- (optional) LLMs: Anbindung von Modellen wie GPT, Mistral oder Claude für Textgenerierung und Dialoge

In dieser Übung konzentrieren wir uns ausschließlich auf den Retrieval-Teil – also das Chunken, Erstellen von Embeddings und semantische Suchen, nicht auf die Textgenerierung.

## 0. Installation benötigter Bibliotheken

- Aktivieren Sie Ihre `venv`
- Aktivieren Sie die benötigten Bibliotheken: `pip install -U langchain langchain-community langchain-text-splitters langchain-huggingface chromadb sentence-transformers torch arxiv tqdm`
- Aktualisieren Sie Ihre `requirements.txt`
- Nach der Installation müssen Sie den Kernel des Notebooks ggf. neu starten, damit die neuen Bibliotheken verfügbar sind


## 1. Data Collection mit der arXiv API

1. Überlegen Sie sich einen Suchbegriff und definieren Sie eine entsprechende Variable. 
2. Überlegen Sie, wieviele Treffer Sie erhalten möchten und definieren Sie eine entsprechende Variable. 
3. Übergeben Sie die beiden Variablen als Parameter `query` und `max_results` an das untenstehende Search()-Objekt.

In [1]:
import arxiv
from tqdm import tqdm

# ...
# ...
# search = arxiv.Search(..., ..., sort_by=arxiv.SortCriterion.Relevance)

4. Führen Sie die Suche aus und lassen Sie sich die Trefferliste mit dem vorhandenen Code ausgeben. 
5. Bonus: Ändern Sie die Sortierung der Ergebnisse von `Relevance` auf `SubmittedDate` und führen Sie die Suche erneut aus.

In [ ]:
docs = []
for result in search.results():
    docs.append({
        "title": result.title,
        "summary": result.summary,
        "url": result.entry_id,
        "published": result.published
    })

print(f"{len(docs)} Artikel geladen.")
for d in docs[:2]:
    print(f"\n📄 {d['title']}\n{d['summary'][:250]}...")

/var/folders/j9/lnrs75px2vvdhgh2j143swp40000gn/T/ipykernel_10477/1247162351.py:3: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


10 Artikel geladen.

📄 Trustworthy and Efficient LLMs Meet Databases
In the rapidly evolving AI era with large language models (LLMs) at the core,
making LLMs more trustworthy and efficient, especially in output generation
(inference), has gained significant attention. This is to reduce plausible but
faulty LLM output...

📄 Large Language Models as Software Components: A Taxonomy for LLM-Integrated Applications
Large Language Models (LLMs) have become widely adopted recently. Research
explores their use both as autonomous agents and as tools for software
engineering. LLM-integrated applications, on the other hand, are software
systems that leverage an LLM t...


## 2. Chunking

1. Betrachten Sie den folgenden Code: Welche Chunking-Methode wird hier eingesetzt? 
2. Passen Sie den Code so an, dass Sliding Window berücksichtigt wird. 
3. Experimentieren Sie mit verschiedenen Werten für die Größe der Chunks und des Sliding Window. 

In [16]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.documents import Document

# In LangChain-Dokumente umwandeln
documents = [
    Document(page_content=d["summary"], metadata={"title": d["title"], "url": d["url"]})
    for d in docs
]

splitter = CharacterTextSplitter(
    separator = ' ',
    chunk_size=100, 
    chunk_overlap=0
    )
chunks = splitter.split_documents(documents)

print(f"{len(chunks)} Chunks erzeugt.")

for i, c in enumerate(chunks[:3]):
    print(f"\n--- Chunk {i+1} ---")
    print(c.page_content)

134 Chunks erzeugt.

--- Chunk 1 ---
In the rapidly evolving AI era with large language models (LLMs) at the core,
making LLMs more

--- Chunk 2 ---
trustworthy and efficient, especially in output generation
(inference), has gained significant

--- Chunk 3 ---
attention. This is to reduce plausible but
faulty LLM outputs (a.k.a hallucinations) and meet the


## 3. Embeddings erzeugen und Vektordatenbank erstellen
1. Wählen Sie ein HggingFace-Embedding-Modell aus (z.B. eines von denen, die wir uns bereits gemeinsam angeschaut haben). 
2. Nutzen Sie das Modell, um Embeddings für die Chunks zu erzeugen und eine Vektordatenbank zu erstellen.

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# model_name = ... 
embeddings = HuggingFaceEmbeddings(model_name=model_name)

db = Chroma.from_documents(chunks, embedding=embeddings, persist_directory="../../data/arxiv_db")

print("Vektordatenbank erstellt.")

/Users/elisabeth/repos/rag_course/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Vektordatenbank erstellt.


## 4. Semantische Suche in der Vektordatenbank
1. Überlegen Sie sich eine Frage oder Phrase, mit der Sie nach ähnlichen Dokumenten in der Vektordatenbank suchen und schauen Sie sich die Ergebnisse an. 
2. Experimentieren Sie mit verschiedenen Queries und Werten für `k`. 
3. Wie könnte man Metadaten (z. B. Jahr, Autor:innen) für die Suche nutzen? 


In [ ]:
# query = ...
# k = ...
results = db.similarity_search(query, k)

for hit in results:
    print(f"\n Treffer: {hit.metadata['title']}")
    print(hit.page_content)
    print("Quelle:", hit.metadata["url"])



 Treffer: A Survey of Large Language Models for Code: Evolution, Benchmarking, and Future Trends
performance
compared to general LLMs. We aim to address three questions: (1) What LLMs
Quelle: http://arxiv.org/abs/2311.10372v2

 Treffer: A Survey of Large Language Models for Code: Evolution, Benchmarking, and Future Trends
performance
compared to general LLMs. We aim to address three questions: (1) What LLMs
Quelle: http://arxiv.org/abs/2311.10372v2

 Treffer: A Survey of Large Language Models for Code: Evolution, Benchmarking, and Future Trends
performance
compared to general LLMs. We aim to address three questions: (1) What LLMs
Quelle: http://arxiv.org/abs/2311.10372v2


## 5. Erweiterungen

### 1. Mehr Daten
- Holen Sie sich eine größere Menge an Dokumenten (z.B. `max_results=100`) und erstellen Sie damit eine Vektordatenbank. 
- Holen Sie sich Artikel zu **verschiedenen** Suchbegriffen und kombinieren Sie diese in einer Vektordatenbank. 

### 2. Andere Chunking-Methode
- Informieren Sie sich über andere Möglichkeiten des Chunkings mit Hilfe der LangChain-Docs: https://docs.langchain.com/oss/python/integrations/splitters
- Probieren Sie mindestens eine weiter Chunking-Methode aus und analysieren Sie, was sich verändert. 
- Welche Methode würdem Sie für arXiv-Abstracts empfehlen und warum?
- Wäre es auch eine Möglichkeit, gar nicht zu chunken? Begründen Sie.  

### 3. Verfeinerung der Suche
- Filtern Sie die Ergebnisse nach Erscheinungsjahr